In [32]:
import urllib.parse as up
import urllib.request as urlo
import json
import gzip
from io import BytesIO
#import babelnet as bn
#print(stopwords.words('italian'))

In [33]:
WORDS_1 = ['ammazzare', 'ingannare', 'pioggia', 'mele', 'spazio']

### BABELNET

In [34]:
def get_indexes_by_word(word):
    service_url = 'https://babelnet.io/v6/getSynsetIds'

    lemma = word
    lang = 'IT'
    key  = 'd4bc0872-d933-4516-b456-2063fa7d68ff'

    params = {
            'lemma' : lemma,
            'searchLang' : lang,
            'key'  : key
    }

    url = service_url + '?' + up.urlencode(params)
    request = urlo.Request(url)
    request.add_header('Accept-encoding', 'gzip')
    response = urlo.urlopen(request)

    if response.info().get('Content-Encoding') == 'gzip':
            buf = BytesIO( response.read())
            f = gzip.GzipFile(fileobj=buf)
            data = json.loads(f.read())
            for result in data:
                return result['id']

In [35]:
def get_synset_by_id(id):
    service_url = 'https://babelnet.io/v6/getSynset'

    key  = 'd4bc0872-d933-4516-b456-2063fa7d68ff'

    params = {
        'id' : id,
        'key'  : key,
        'targetLang' : 'IT'
    }

    url = service_url + '?' + up.urlencode(params)

    request = urlo.Request(url)
    request.add_header('Accept-encoding', 'gzip')
    response = urlo.urlopen(request)
    
    if response.info().get('Content-Encoding') == 'gzip':
        buf = BytesIO(response.read())
        f = gzip.GzipFile(fileobj=buf)
        data = json.loads(f.read())

        # retrieving BabelSense data
        senses = data['senses']
        for result in senses:
            lemma = result.get('lemma')
            language = result.get('language')
            if language != None:    
                print(str(language.encode('utf-8')) + "\t" + str(lemma.encode('utf-8')))

        print('\n')
        # retrieving BabelGloss data
        glosses = data['glosses']
        for result in glosses:
            gloss = result.get('gloss')
            language = result.get('language')
            if language != None:
                print(str(language.encode('utf-8')) + "\t" + str(gloss.encode('utf-8')))

In [36]:
def get_hype_hypo_anto(id):
    service_url = 'https://babelnet.io/v6/getOutgoingEdges'

    key  = 'd4bc0872-d933-4516-b456-2063fa7d68ff'

    params = {
        'id' : id,
        'key'  : key,
        'targetLang' : 'IT'
    }

    url = service_url + '?' + up.urlencode(params)

    request = urlo.Request(url)
    request.add_header('Accept-encoding', 'gzip')
    response = urlo.urlopen(request)

    if response.info().get('Content-Encoding') == 'gzip':
        buf = BytesIO( response.read())
        f = gzip.GzipFile(fileobj=buf)
        data = json.loads(f.read())

        # retrieving Edges data
        for result in data:
            target = result.get('target')
            language = result.get('language')

            # retrieving BabelPointer data
            pointer = result['pointer']
            relation = pointer.get('name')
            group = pointer.get('relationGroup')

            # Types of relationGroup: HYPERNYM,  HYPONYM, MERONYM, HOLONYM, OTHER
            if 'hypernym' in group.lower() or 'hyponym' in group.lower():
                if get_synset_by_id(target) != None:
                    print(f"{get_synset_by_id(target)}   {str(relation)}   {str(group)}")
            elif 'antonym' in relation.lower():
                if get_synset_by_id(target) != None:
                    print(f"{get_synset_by_id(target)}   {str(relation)}   {str(group)}")

In [37]:
def get_all_defs(words):
    senses = []
    for word in words:
        id = get_indexes_by_word(word)
        senses.append(get_synset_by_id(id))
    return senses

In [38]:
words1_senses = get_all_defs(WORDS_1)

HTTPError: HTTP Error 403: Forbidden